In [1]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.cosmology import Planck18
#from scipy.special import erf, erfc,erfcx
import scipy
import emcee
import corner
from astropy.table import Table
from astropy.io import fits
import fitsio
import matplotlib as mpl
from multiprocessing import Pool
import torch
from torch.special import erfcx 


In [2]:
k = Table.read('/pscratch/sd/z/zhaozhon/DATASET/DESI-data/k.csv',format='csv')
indice = np.isnan(k['k_gg']) | np.isinf(k['k_gg']) | np.isnan(k['k_rr']) | np.isinf(k['k_rr'])
k_gg = k['k_gg'][~indice]
k_rr = k['k_rr'][~indice]

In [3]:
file = '/pscratch/sd/z/zhaozhon/DATASET/DESI-data/new_d.fits'
d = Table(fitsio.read(file))
SEL = np.where((d['Z'] > 2.1) & (d['Z'] < 2.8))[0]
apr_mag_g = d['APP_MAG_G'][SEL][~indice]
apr_mag_g_err = d['APP_MAGERR_G'][SEL][~indice]
apr_mag_r = d['APP_MAG_R'][SEL][~indice]
apr_mag_r_err = d['APP_MAGERR_R'][SEL][~indice]
z = d['Z'][SEL][~indice]
TARGETID = d['TARGETID'][SEL][~indice]
d = Planck18.distmod(z).value

In [4]:
m_lim = 22.5
indice = (apr_mag_g < m_lim) & (apr_mag_r < m_lim)
apr_mag_g = apr_mag_g[indice]
apr_mag_g_err = apr_mag_g_err[indice]
d = d[indice]
k_gg = k_gg[indice]

apr_mag_r = apr_mag_r[indice]
apr_mag_r_err = apr_mag_r_err[indice]
k_rr = k_rr[indice]

In [5]:
def log_likelihood(params, apr_mag1, magerr1, k1, apr_mag2, magerr2, k2,d, m_lim):
    sigma_I1,M_X1,a1 = params[:3]
    sigma_I2, M_X2,a2 = params[3:6]
    #m_real1 = params[5:apr_mag1.shape[0]+5]
    #m_real2 = params[apr_mag1.shape[0]+5:2*apr_mag1.shape[0]+5]
    #A_x = params[2*apr_mag1.shape[0]+5:3*apr_mag1.shape[0]+5]
    A_x = params[6:apr_mag1.shape[0]+6]

    #term1 = -torch.log(2 * torch.pi * magerr1 * sigma_I1)  # Avoid division by zero
    #term1 = -torch.log(magerr1)-torch.log(sigma_I1)
    term1 = -torch.log(sigma_I1) - 0.5*torch.log(1+apr_mag1**2/sigma_I1**2)
    #term1 = -0.5*torch.log(sigma_I1**2+apr_mag1**2)
    #term2 = -0.5 * ((apr_mag1 - m_real1) / magerr1) ** 2
    term3 = -0.5 * (apr_mag1 - d - k1 - M_X1 + a1*A_x)**2 / (magerr1**2 + sigma_I1**2)
    #term3 = -0.5 * ((m_real1 - d - k1 - M_X1 + A_x) / sigma_I1) ** 2
    #term4 = -torch.log(torch.tensor(0.5)) - torch.log(torch.erfc((apr_mag1 - m_lim) / (torch.sqrt(torch.tensor(2.0)) * magerr1)))
    
    #term5 = -torch.log(2 * torch.pi * magerr2 * sigma_I2)  # Avoid division by zero
    #term5 =  -torch.log(magerr2)-torch.log(sigma_I2)
   # term5 = -0.5*torch.log(2 * torch.pi * (magerr2**2 + sigma_I2**2))
    term5 = -torch.log(sigma_I2) - 0.5*torch.log(1+apr_mag2**2/sigma_I2**2)
    #term5 = -0.5*torch.log(sigma_I2**2+apr_mag2**2)
    #term6 = -0.5 * ((apr_mag2 - m_real2) / magerr2) ** 2
    term7 = -0.5 * ((apr_mag2 - d - k2 - M_X2 + a2*A_x)**2 / (magerr2**2 + sigma_I2**2))
    #term7 = -0.5 * ((m_real2 - d - k2 - M_X2 + A_x) / sigma_I2) ** 2
    #term8 = -torch.log(torch.tensor(0.5)) - torch.log(torch.erfc((apr_mag2 - m_lim) / (torch.sqrt(torch.tensor(2.0)) * magerr2)))
    #term9 = -0.5*(A_x**2)
    #print(torch.sum(term3))
    total_log_likelihood = torch.sum(term1+term3 + term5 + term7)# + term9)
    return total_log_likelihood

In [6]:
apr_mag_g = apr_mag_g.astype(np.float32)
apr_mag_g_err = apr_mag_g_err.astype(np.float32)
k_gg = k_gg.astype(np.float32)

apr_mag_r = apr_mag_r.astype(np.float32)
apr_mag_r_err = apr_mag_r_err.astype(np.float32)
k_rr = k_rr.astype(np.float32)

d = d.astype(np.float32)

# Convert your data to PyTorch tensors
apr_mag_g_tensor = torch.from_numpy(apr_mag_g)
apr_mag_g_err_tensor = torch.from_numpy(apr_mag_g_err)
k_gg_tensor = torch.from_numpy(k_gg)

apr_mag_r_tensor = torch.from_numpy(apr_mag_r)
apr_mag_r_err_tensor = torch.from_numpy(apr_mag_r_err)
k_rr_tensor = torch.from_numpy(k_rr)

d_tensor = torch.from_numpy(d)

In [7]:
import numpy as np
from scipy.stats import dirichlet

def sample_simplex_centered(N):
    # Sample B from a symmetric Dirichlet distribution (Simplex)
    B = dirichlet.rvs([1]*N)[0]  # rvs generates samples, [1]*N makes it uniform

    # Compute A such that the mean of A is zero
    A = B - 1/N
    
    return B, A
B, A = sample_simplex_centered(apr_mag_g_tensor.shape[0])
A_tensor = torch.from_numpy(A)

In [ ]:
params = torch.zeros(apr_mag_g_tensor.shape[0]+6, requires_grad=False)
params[0] = 0.5
params[1] = -24
params[2] = 1.5
params[3] = 0.5
params[4] = -24
params[5] = 1.5
params[6:apr_mag_g_tensor.shape[0]+6] = A_tensor

# Adjust requires_grad to True afterwards
params.requires_grad_(True)

optimizer = torch.optim.AdamW([params], lr=0.001)
epochs = []
Losses = []
param0_values = []  # List to track params[0] values
param1_values = []  # List to track params[1] values
param2_values = []
param3_values = []  # List to track params[0] values
param4_values = []  # List to track params[1] values
param5_values = []
param6_values = []
A1 = []
# Optimization loop
for epoch in range(1000000):  # Number of epochs, adjust as needed
    optimizer.zero_grad()
    loss = -log_likelihood(params, apr_mag_g_tensor, apr_mag_g_err_tensor, k_gg_tensor,apr_mag_r_tensor,
                           apr_mag_r_err_tensor, k_rr_tensor,d_tensor,22.5 )
    loss.backward()
    optimizer.step()

    subset_start_g = 6
    subset_end_g = apr_mag_g_tensor.shape[0] + 6
    subset_mean_g = params[subset_start_g:subset_end_g].mean()
    params.data[subset_start_g:subset_end_g] -= subset_mean_g
    
    #with torch.no_grad():
        #params[0].clamp_(0.01,1000)#intrinsic dispersion
        #params[1].clamp_(-30,-20)#average absolute magnitude
        #params[2].clamp_(-3,3)
        #params[3].clamp_(0.01,1000)#intrinsic dispersion
        #params[4].clamp_(-30,-20)#average absolute magnitude
        #params[5].clamp_(-3,3)
        #params[4].clamp_(-1,1)
        #params[subset_start_g:subset_end_g].clamp_(-0.5,0.5)
        
    epochs.append(epoch)
    Losses.append(loss.item())
    param0_values.append(params[0].item())
    param1_values.append(params[1].item())
    param2_values.append(params[2].item())
    param3_values.append(params[3].item())
    param4_values.append(params[4].item())
    param5_values.append(params[5].item())
    param6_values.append(params[6].item())
    if epoch % 100 == 0:  # Print loss every 100 epochs
        print(f"Epoch {epoch}, Loss: {loss.item()}, Param0: {params[0].item()}, Param1: {params[1].item()},Params2:{params[2].item()},Params3:{params[3].item()},Params4:{params[4].item()},Params5:{params[5].item()},A1:{params[6].item()}")

Epoch 0, Loss: 1734435.5, Param0: 0.5009949803352356, Param1: -23.998760223388672,Params2:1.498984932899475,Params3:0.5009949803352356,Params4:-24.00075912475586,Params5:1.498984932899475,A1:-0.0009175220038741827
Epoch 100, Loss: 1451780.0, Param0: 0.5871685743331909, Param1: -23.98613166809082,Params2:1.625976324081421,Params3:0.5871119499206543,Params4:-24.056711196899414,Params5:1.6260257959365845,A1:-0.07940632104873657
Epoch 200, Loss: 1317567.125, Param0: 0.6466731429100037, Param1: -23.986032485961914,Params2:1.7586199045181274,Params3:0.6466826796531677,Params4:-24.0759220123291,Params5:1.7589221000671387,A1:-0.1310815066099167
Epoch 300, Loss: 1240569.25, Param0: 0.689103364944458, Param1: -23.985977172851562,Params2:1.877991795539856,Params3:0.6893700957298279,Params4:-24.081499099731445,Params5:1.8787906169891357,A1:-0.16068269312381744
Epoch 400, Loss: 1192481.75, Param0: 0.7197281718254089, Param1: -23.985942840576172,Params2:1.9839282035827637,Params3:0.720368504524231,P

In [ ]:
'''
# Initialize parameters, ensuring they require gradients
params1 = torch.randn(2*apr_mag_g.shape[0]+2, requires_grad=True)  # Adjust size according to your total number of parameters
params2 = torch.randn(2*apr_mag_r.shape[0]+2, requires_grad=True)

# Define optimizer
optimizer = torch.optim.Adamax([params1, params2], lr=0.005)


epochs = []
Losses = []
# Optimization loop
for epoch in range(150000):  # Number of epochs, adjust as needed
    optimizer.zero_grad()
    loss = -log_likelihood(params1, apr_mag_g_tensor, apr_mag_g_err_tensor, d_g_tensor, k_gg_tensor,22.5)-log_likelihood(
        params2, apr_mag_r_tensor, apr_mag_r_err_tensor, d_r_tensor, k_rr_tensor,22.5 )
    loss.backward()
    optimizer.step()
    
    # Adjust params1 subset to have an average of 0
    subset_start_g = apr_mag_g_tensor.shape[0] + 3
    subset_end_g = 2 * apr_mag_g_tensor.shape[0] + 3
    subset_mean_g = params1[subset_start_g:subset_end_g].mean()
    params1.data[subset_start_g:subset_end_g] -= subset_mean_g
    
    # Adjust params2 subset to have an average of 0
    subset_start_r = apr_mag_r_tensor.shape[0] + 3
    subset_end_r = 2 * apr_mag_r_tensor.shape[0] + 3
    subset_mean_r = params2[subset_start_r:subset_end_r].mean()
   # params2.data[subset_start_r:subset_end_r] -= subset_mean_r
    with torch.no_grad():
        params1[0].clamp_(0.001,1)#intrinsic dispersion
        params1[1].clamp_(-30,-20)#average absolute magnitude
        params2[0].clamp_(0.001,1)#intrinsic dispersion
        params2[1].clamp_(-30,-20)#average absolute magnitude
        #params[apr_mag_g.shape[0]+2:2*apr_mag_g.shape[0]+2].clamp_(0,10)#extinction of X
        #params[2*apr_mag_g.shape[0]+2:3*apr_mag_g.shape[0]+2].clamp_(0,10)#extinction o Y
 #       params[apr_mag_g.shape[0]+2:].clamp_(0,1000000000)
    epochs.append(epoch)
    Losses.append(loss.item())
    if epoch % 100 == 0:  # Print loss every 100 epochs
        print(f"Epoch {epoch}, Loss: {loss.item()}")
        '''

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(epochs, Losses, label='Loss', color='blue')  # Plot Losses vs. epochs
plt.title('Loss vs. Epochs')  # Title of the plot
plt.xlabel('Epochs')  # Label for the x-axis
plt.ylabel('Loss')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Show grid lines for better readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(epochs, param0_values, label='$sigma_I$', color='blue')  # Plot Losses vs. epochs
plt.title('Intrinsic dispersion for g band vs. Epochs')  # Title of the plot
plt.xlabel('Epochs')  # Label for the x-axis
plt.ylabel('$\sigma_I$')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Show grid lines for better readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(epochs, param3_values, label='$sigma_I$', color='blue')  # Plot Losses vs. epochs
plt.title('Intrinsic dispersion for r band vs. Epochs')  # Title of the plot
plt.xlabel('Epochs')  # Label for the x-axis
plt.ylabel('$\sigma_I$')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Show grid lines for better readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(epochs, param1_values, label='M', color='blue')  # Plot Losses vs. epochs
plt.title('Absolute Magnitude for g band vs. Epochs')  # Title of the plot
plt.xlabel('Epochs')  # Label for the x-axis
plt.ylabel('M')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Show grid lines for better readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(epochs, param4_values, label='M', color='blue')  # Plot Losses vs. epochs
plt.title('Absolute Magnitude for r band vs. Epochs')  # Title of the plot
plt.xlabel('Epochs')  # Label for the x-axis
plt.ylabel('M')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Show grid lines for better readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(epochs, param2_values, label='a_g', color='blue')  # Plot Losses vs. epochs
plt.title('a_g for g band vs. Epochs')  # Title of the plot
plt.xlabel('Epochs')  # Label for the x-axis
plt.ylabel('a_g')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Show grid lines for better readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(epochs, param5_values, label='a_r', color='blue')  # Plot Losses vs. epochs
plt.title('a_r for r band vs. Epochs')  # Title of the plot
plt.xlabel('Epochs')  # Label for the x-axis
plt.ylabel('a_r')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Show grid lines for better readability
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(epochs, param6_values, label='A_x[0]', color='blue')  # Plot Losses vs. epochs
plt.title('A_x[0] vs. Epochs')  # Title of the plot
plt.xlabel('Epochs')  # Label for the x-axis
plt.ylabel('A_x[0]')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Show grid lines for better readability
plt.show()

In [ ]:
# Access optimized parameters
optimized_params = params.data

# If you need to convert to a numpy array for further processing outside PyTorch
optimized_params_numpy = optimized_params.numpy()

print("Optimized Parameters:", optimized_params)
print("As Numpy Array:", optimized_params_numpy)

In [ ]:
plt.hist(optimized_params_numpy[6:apr_mag_g_tensor.shape[0]+6],bins=1000)